# 🏗️ HVAC AI Platform - One-Click Cloud Launcher

**Repository:** `https://github.com/elliotttmiller/hvac.git`

This notebook provides instant cloud deployment of the HVAC AI Platform on Google Colab.

---

## 📋 Features
- ✅ Automated environment setup (Node.js 20+ LTS, Python 3.11+)
- ✅ Repository cloning and dependency installation
- ✅ Secure API key configuration
- ✅ Public URL tunneling (localtunnel) for frontend & backend
- ✅ Full platform launch via `start.py`

---

## 🔐 Required Secrets
Before running, you'll need:
- `VITE_AI_API_KEY` or `VITE_GEMINI_API_KEY` - Your AI API key

---

## 🚀 Quick Start
1. **Run all cells sequentially** (Runtime → Run all)
2. **Enter your API keys** when prompted
3. **Access your app** via the printed tunnel URLs

---

## 🔧 Step 1: Mount Google Drive (Optional)

Mount your Google Drive to persist data and configurations.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive', force_remount=False)

print("✅ Google Drive mounted successfully")
print(f"📂 Current directory: {os.getcwd()}")

## 🐍 Step 2: Verify Environment

In [ ]:
%%bash

# Install Node.js 20.x LTS from NodeSource
echo "📦 Installing Node.js 20.x LTS..."

# Check if Node.js is already installed
if command -v node &> /dev/null; then
    CURRENT_VERSION=$(node --version)
    echo "ℹ️  Node.js already installed: $CURRENT_VERSION"

    # Check if version is 20+
    MAJOR_VERSION=$(echo $CURRENT_VERSION | cut -d'v' -f2 | cut -d'.' -f1)
    if [ "$MAJOR_VERSION" -ge 20 ]; then
        echo "✅ Node.js version is sufficient (v20+)"
    else
        echo "⚠️  Node.js version is too old, upgrading..."
        curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
        sudo apt-get install -y nodejs
        echo "✅ Node.js upgraded successfully"
    fi
else
    echo "ℹ️  Node.js not found, installing..."
    curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
    sudo apt-get install -y nodejs
    echo "✅ Node.js installed successfully"
fi

# Verify installation
echo "✅ Node.js version:"
node --version
echo "✅ npm version:"
npm --version

## 📥 Step 4: Clone HVAC Repository

Clone the repository from GitHub.

In [ ]:
import os
import shutil

# Define repo details
REPO_URL = "https://github.com/elliotttmiller/hvac.git"
REPO_DIR = "/content/hvac"

# Remove existing directory if present
if os.path.exists(REPO_DIR):
    print(f"⚠️  Removing existing directory: {REPO_DIR}")
    shutil.rmtree(REPO_DIR)

# Clone repository
print(f"📥 Cloning repository from {REPO_URL}...")
!git clone {REPO_URL} {REPO_DIR}

# Change to repo directory
os.chdir(REPO_DIR)

print(f"✅ Repository cloned successfully")
print(f"📂 Current directory: {os.getcwd()}")

# Show directory contents
print("\n📁 Repository contents:")
!ls -la

## 🔐 Step 5: Configure API Keys

**CRITICAL:** Enter your API keys when prompted. These will be injected into `.env` file.

You can get API keys from:
- **Gemini:** https://makersuite.google.com/app/apikey
- **OpenAI:** https://platform.openai.com/api-keys

In [ ]:
from google.colab import userdata

# Attempt to load secrets using userdata.get
print("🔐 API Key Configuration")
print("=" * 50)

# Default to Colab secrets
ai_provider = userdata.get('VITE_AI_PROVIDER') or "gemini"
api_key = userdata.get('VITE_AI_API_KEY') or ""
gemini_key = userdata.get('VITE_GEMINI_API_KEY') or ""

# Fallback to manual input if keys are missing
if not api_key:
    print("\n⚠️  API key not found in Colab secrets. Falling back to manual input.")
    ai_provider = input("\nSelect AI Provider (gemini/openai/anthropic) [gemini]: ").strip().lower() or "gemini"
    if ai_provider == "gemini":
        api_key = getpass("Enter your GEMINI API Key: ")
        gemini_key = api_key
    elif ai_provider == "openai":
        api_key = getpass("Enter your OPENAI API Key: ")
        gemini_key = ""
    else:
        api_key = getpass("Enter your API Key: ")
        gemini_key = ""

    # Optional: Gemini-specific key
    if not gemini_key:
        gemini_prompt = input("\nAlso provide Gemini key? (y/n) [n]: ").strip().lower()
        if gemini_prompt == 'y':
            gemini_key = getpass("Enter your GEMINI API Key: ")

# Generate .env file
env_content = f"""# ============================================================================
# AI Provider Configuration (Auto-generated by Colab Launcher)
# ============================================================================

# AI Provider Selection
VITE_AI_PROVIDER={ai_provider}

# AI API Key
VITE_AI_API_KEY={api_key}

# Provider-specific keys
{f"VITE_GEMINI_API_KEY={gemini_key}" if gemini_key else "# VITE_GEMINI_API_KEY=your_gemini_key_here"}

# Model Selection
VITE_AI_MODEL={"gemini-2.5-flash" if ai_provider == "gemini" else "gpt-4o" if ai_provider == "openai" else "claude-3-5-sonnet-20241022"}

# AI Generation Parameters
VITE_AI_TEMPERATURE=0.2
VITE_AI_MAX_TOKENS=4096

# ============================================================================
# Feature Flags
# ============================================================================

VITE_FEATURE_CACHE=true
VITE_FEATURE_FILE_PROCESSING=true

# ============================================================================
# Rate Limiting
# ============================================================================

VITE_RATE_LIMIT_MAX_RETRIES=3
VITE_RATE_LIMIT_DELAY_MS=1000
VITE_RATE_LIMIT_EXPONENTIAL_BACKOFF=true

# ============================================================================
# File Processing
# ============================================================================

VITE_FILE_MAX_SIZE=10485760
VITE_FILE_SUPPORTED_FORMATS=pdf,png,jpg,jpeg,dwg
VITE_FILE_PDF_DPI=300
"""

# Write .env file
env_path = "/content/hvac/.env"
with open(env_path, "w") as f:
    f.write(env_content)

print("\n✅ .env file created successfully")
print(f"📄 Location: {env_path}")
print("\n⚠️  API keys are sensitive - never commit .env to version control!")

## 📦 Step 6: Install Dependencies

Install npm dependencies for the project.

In [ ]:
%%bash

cd /content/hvac

echo "📦 Installing npm dependencies..."
echo "⏱️  This may take 2-3 minutes..."

npm install

echo "✅ Dependencies installed successfully"

## 🚀 Step 8: Launch Application

**IMPORTANT:** This cell will start the application servers and create public URLs.

The cell will:
1. Run `start.py` to validate environment and start dev servers
2. Create public tunnels for frontend (port 3000) and backend (port 4000)
3. Print public URLs that you can access from any browser

**Note:** The servers will keep running. To stop them, use Runtime → Interrupt execution.

In [ ]:
import subprocess
import threading
import time
import os
import requests
import socket
import sys
import re

# Configuration
FRONTEND_PORT = 3000
BACKEND_PORT = 4000
PROJECT_ROOT = '/content/hvac'

# ANSI Colors
class Colors:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

print(f"{Colors.HEADER}🚀 Initializing HVAC AI Platform (Cloudflare Edition)...{Colors.ENDC}", flush=True)
print("=" * 70, flush=True)

# 1. Install Cloudflare Tunnel (cloudflared)
print("📦 Installing Cloudflare Tunnel...", flush=True)
if not os.path.exists("cloudflared"):
    subprocess.run(
        ["wget", "-q", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"],
        check=True
    )
    subprocess.run(["mv", "cloudflared-linux-amd64", "cloudflared"], check=True)
    subprocess.run(["chmod", "+x", "cloudflared"], check=True)
    print(f"{Colors.GREEN}✅ cloudflared installed{Colors.ENDC}", flush=True)
else:
    print(f"{Colors.GREEN}✅ cloudflared already present{Colors.ENDC}", flush=True)

# Ensure directory exists
if not os.path.exists(PROJECT_ROOT):
    PROJECT_ROOT = os.getcwd()
os.chdir(PROJECT_ROOT)

# 2. Cleanup Old Processes
def kill_process_on_port(port):
    try:
        subprocess.run(["fuser", "-k", f"{port}/tcp"], capture_output=True)
    except Exception:
        pass

print("🔍 Pre-flight check: Cleaning ports...", flush=True)
kill_process_on_port(FRONTEND_PORT)
kill_process_on_port(BACKEND_PORT)

# 3. Log Streamer
def stream_logs(process, prefix, color):
    try:
        for line in iter(process.stdout.readline, ''):
            if line:
                print(f"{color}[{prefix}] {line.strip()}{Colors.ENDC}", flush=True)
    except (ValueError, OSError):
        pass

# 4. Run Diagnostics
if os.path.exists('start.py'):
    print("\n🩺 Running Platform Diagnostics...", flush=True)
    diag_proc = subprocess.run(['python3', 'start.py', '--no-dev'], capture_output=True, text=True)
    if diag_proc.returncode != 0:
        print(f"{Colors.FAIL}❌ Diagnostics Failed:{Colors.ENDC}", flush=True)
        print(diag_proc.stdout, flush=True)
    else:
        print(f"{Colors.GREEN}✅ Environment Diagnostics Passed{Colors.ENDC}", flush=True)

# 5. Launch Servers
print(f"\n{Colors.BOLD}⚡ Launching Services...{Colors.ENDC}", flush=True)

env_vars = os.environ.copy()
env_vars['FORCE_COLOR'] = 'true'

frontend_proc = subprocess.Popen(['npm', 'run', 'dev'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True, env=env_vars)
backend_proc = subprocess.Popen(['npm', 'run', 'dev:api'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True, env=env_vars)

# Start Log Threads
t_front = threading.Thread(target=stream_logs, args=(frontend_proc, "Frontend", Colors.CYAN))
t_front.daemon = True; t_front.start()

t_back = threading.Thread(target=stream_logs, args=(backend_proc, "Backend ", Colors.BLUE))
t_back.daemon = True; t_back.start()

# 6. Wait for Ports
def wait_for_port(port, timeout=60):
    start_time = time.time()
    while time.time() - start_time < timeout:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            if sock.connect_ex(("localhost", port)) == 0:
                return True
        time.sleep(1)
    return False

print("⏳ Waiting for servers...", flush=True)
if wait_for_port(FRONTEND_PORT) and wait_for_port(BACKEND_PORT):
    print(f"{Colors.GREEN}✅ Servers listening!{Colors.ENDC}", flush=True)
else:
    print(f"{Colors.FAIL}❌ Timeout waiting for servers.{Colors.ENDC}", flush=True)

# 7. Start Cloudflare Tunnels
tunnel_urls = {}

def start_cf_tunnel(port, name):
    try:
        # Start cloudflared
        proc = subprocess.Popen(
            [f"{PROJECT_ROOT}/cloudflared", "tunnel", "--url", f"http://localhost:{port}"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            universal_newlines=True
        )

        # Parse output for the URL
        for line in iter(proc.stdout.readline, ''):
            if '.trycloudflare.com' in line:
                # Extract URL using regex
                match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
                if match:
                    url = match.group(0)
                    tunnel_urls[name] = url
                    return # Stop reading once we have the URL
    except Exception as e:
        print(f"Tunnel Error: {e}")

print("🌐 Establishing Cloudflare Tunnels (No Password Required)...", flush=True)
t_cf_front = threading.Thread(target=start_cf_tunnel, args=(FRONTEND_PORT, 'Frontend'))
t_cf_back = threading.Thread(target=start_cf_tunnel, args=(BACKEND_PORT, 'Backend'))

t_cf_front.daemon = True; t_cf_front.start()
t_cf_back.daemon = True; t_cf_back.start()

# Wait for URLs to appear
time.sleep(8)

# 8. Final Dashboard
print("\n" + "=" * 70, flush=True)
print(f"{Colors.GREEN}{Colors.BOLD}🎉 HVAC AI PLATFORM ONLINE{Colors.ENDC}", flush=True)
print("=" * 70, flush=True)

print(f"{Colors.CYAN}🌍 PUBLIC ACCESS (No Password Needed):{Colors.ENDC}", flush=True)
print(f"   Frontend: {Colors.BOLD}{tunnel_urls.get('Frontend', 'Initializing...')}{Colors.ENDC}", flush=True)
print(f"   Backend:  {Colors.BOLD}{tunnel_urls.get('Backend', 'Initializing...')}{Colors.ENDC}", flush=True)
print("", flush=True)

print(f"{Colors.YELLOW}🏠 LOCAL ACCESS:{Colors.ENDC}", flush=True)
print(f"   Frontend: http://localhost:{FRONTEND_PORT}", flush=True)
print("-" * 70, flush=True)
print(f"{Colors.BOLD}📝 LIVE SERVER LOGS:{Colors.ENDC}", flush=True)
print("-" * 70, flush=True)

# 9. Keep Alive
try:
    while True:
        time.sleep(1)
        if frontend_proc.poll() is not None:
            print(f"{Colors.FAIL}❌ Frontend died{Colors.ENDC}", flush=True); break
        if backend_proc.poll() is not None:
            print(f"{Colors.FAIL}❌ Backend died{Colors.ENDC}", flush=True); break
except KeyboardInterrupt:
    print("\n🛑 Shutting down...", flush=True)
    frontend_proc.terminate()
    backend_proc.terminate()